In [ ]:
!pip install kSpider2 kProcessor

In [ ]:
%%bash
KSIZE=25

VER38_GRCh38_URL=http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_38/gencode.v38.transcripts.fa.gz
VER28_GRCh37_URL=http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_28/GRCh37_mapping/gencode.v28lift37.transcripts.fa.gz

VER38_GRCh38=VER38_GRCh38.fa
VER28_GRCh37=VER28_GRCh37.fa

wget --quiet ${VER38_GRCh38_URL} -O ${VER38_GRCh38}.gz
wget --quiet ${VER28_GRCh37_URL} -O ${VER28_GRCh37}.gz

gunzip *gz

In [ ]:
%%bash
VER38_GRCh38=VER38_GRCh38.fa
VER28_GRCh37=VER28_GRCh37.fa

#1 Transform first annotation -------------------------------------------------------------
sed -i 's/^>/>VER38_GRCh38|/' ${VER38_GRCh38}

grep ">" ${VER38_GRCh38} | cut -c2- |  awk -F'|' '{print $0"\tver38_GRCh38."$3}' > ${VER38_GRCh38}.names

#2 Transform second annotation
sed -i 's/^>/>VER28_GRCh37|/' ${VER28_GRCh37}

grep ">" ${VER28_GRCh37} | cut -c2- |  awk -F'|' '{print $0"\tver28_GRCh37."$3}' > ${VER28_GRCh37}.names


In [ ]:
%%bash
VER38_GRCh38=VER38_GRCh38.fa
VER28_GRCh37=VER28_GRCh37.fa

cat ${VER38_GRCh38} ${VER28_GRCh37} > merged.fa
cat ${VER38_GRCh38}.names ${VER28_GRCh37}.names > merged.fa.names

In [ ]:
import kProcessor as kp

kSize = 25
chunkSize = 5000
fasta_file = "merged.fa"

KF = kp.kDataFramePHMAP(kp.KMERS, kp.nonCanonicalInteger_Hasher, {"kSize":kSize})
cKF = kp.index(KF, fasta_file, chunkSize, fasta_file + ".names")
cKF.save("idx_" + fasta_file.replace(".fa",""))

In [ ]:
%%bash
kSpider2 pairwise -i idx_merged

In [ ]:
%%bash
kPro_index="idx_merged"
paste <(tail -n+2 ${kPro_index}.namesMap |cut -d" " -f1)  <(tail -n+2 ${kPro_index}.namesMap |cut -d" " -f2-) > ${kPro_index}.namesMap.tmp
echo "node_id node_name size" | tr ' ' '\t' > ${kPro_index}_nodes_size.tsv
awk 'BEGIN{FS=OFS="\t";}FNR==NR{a[$2]=$3;next;}{if(a[$1]!="")print $0,a[$1]}' ${kPro_index}_kSpider_seqToKmersNo.tsv ${kPro_index}.namesMap.tmp >> ${kPro_index}_nodes_size.tsv
rm ${kPro_index}.namesMap.tmp*

# 2. Annotation of the numerically coded association file
# a) Add items names and no of associated items 
# b) calc jaccard distance and containment ratio for each pair
# c) Additionally, we can filter out those with minimal similarities 
md=20   ## minimum jaccard distance (as a percentage) to keep the record 
mc=80  ## minimum containment ratio (as a percentage) to keep the record 
echo ":START_ID|START_name|START_size|shared_count:int|jDist:float|smPerc:float|END_name|END_size|:END_ID" > ${kPro_index}_relations.csv
awk -v md=$md -v mc=$mc 'BEGIN{FS="\t";S="|";}FNR==NR{a[$1]=$3;b[$1]=$2S$3;next;}{
   g1=a[$2]; g2=a[$3]; min=g1;min=(min < g2 ? min : g2); 
   jDist=$4*100/(g1+g2-$4); smPerc=$4*100/min; 
   if(jDist>md || smPerc>mc)
     printf("%s%s%s%s%s%s%.1f%s%.1f%s%s%s%s\n", $2,S,b[$2],S,$4,S,jDist,S,smPerc,S,b[$3],S,$3)}' \
   ${kPro_index}_nodes_size.tsv <(tail -n+2 ${kPro_index}_kSpider_pairwise.tsv) >> ${kPro_index}_relations.csv

In [ ]:
tr1_tr1 = dict()
tr1_tr2 = dict()

tr2_tr2 = dict()
tr2_tr1 = dict()

tr1_prefix = "ver28_GRCh37."
tr2_prefix = "ver38_GRCh38."

def keepTheGreatest_local(current_gene, new_gene):
    # print(f"checking {current_gene} VS. {new_gene}")
    newGeneName, newGeneJacard, newGeneCont = new_gene
    currentGeneName, currentGeneJacard, currentGeneCont = current_gene
    newGeneScore = newGeneJacard + newGeneCont
    currentGeneScore = currentGeneJacard + currentGeneCont
    return new_gene if newGeneScore > currentGeneScore else current_gene

def keepTheGreatest(current_list, new_gene):
    current_list = current_list.copy()
    newGeneName, newGeneJacard, newGeneCont = new_gene
    newGeneScore = newGeneJacard + newGeneCont
    maxScoreFound = float()
    if not len(current_list):
        return []

    for currentGene in current_list:
        currentGeneName, currentGeneJacard, currentGeneCont = currentGene
        currentGeneScore = currentGeneJacard + currentGeneCont
        maxScoreFound = max(currentGeneScore, maxScoreFound)
    
    if newGeneScore == maxScoreFound:
        current_list.append(new_gene)
    elif newGeneScore > maxScoreFound:
        current_list = [new_gene]
    
    return current_list
        
        

    

file1 = "sample.csv"
file2 = "idx_merged_relations.csv"

with open(file2) as READER:
    next(READER)
    for line in READER:
        line = line.split('|')
        jaccard, containment = float(line[4]), float(line[5])
        gene1, gene2 = line[1], line[6]
        gene1_full = (gene1, jaccard, containment)
        gene2_full = (gene2, jaccard, containment)


        # gene1 & gene2 belongs to tr1
        if tr1_prefix in gene1 and tr1_prefix in gene2:
            if gene1 not in tr1_tr1:
                tr1_tr1[gene1] = [gene2_full]
            else:
                tr1_tr1[gene1] = keepTheGreatest(tr1_tr1[gene1], gene2_full)
        
            if gene2 not in tr1_tr1:
                tr1_tr1[gene2] = [gene1_full]
            else:
                tr1_tr1[gene2] = keepTheGreatest(tr1_tr1[gene2], gene1_full)

                
        
        # gene1 & gene2 belongs to tr2
        elif tr2_prefix in gene1 and tr2_prefix in gene2:
            if gene1 not in tr2_tr2:
                tr2_tr2[gene1] = [gene2_full]
            else:
                tr2_tr2[gene1] = keepTheGreatest(tr2_tr2[gene1], gene2_full)

            if gene2 not in tr2_tr2:
                tr2_tr2[gene2] = [gene1_full]
            else:
                tr2_tr2[gene2] = keepTheGreatest(tr2_tr2[gene2], gene1_full)
        
        elif tr2_prefix in gene1+gene2 and tr1_prefix in gene1+gene2:
            if tr1_prefix in gene1:
                if gene1 not in tr1_tr2:
                    tr1_tr2[gene1] = [gene2_full]
                else:
                    tr1_tr2[gene1] = keepTheGreatest(tr1_tr2[gene1], gene2_full)

                if gene2 not in tr2_tr1:
                    tr2_tr1[gene2] = [gene1_full]
                else:
                    tr2_tr1[gene2] = keepTheGreatest(tr2_tr1[gene2], gene1_full)
                
            else:
                if gene1 not in tr2_tr1:
                    tr2_tr1[gene1] = [gene2_full]
                else:
                    tr2_tr1[gene1] = keepTheGreatest(tr2_tr1[gene1], gene2_full)

                if gene2 not in tr1_tr2:
                    tr1_tr2[gene2] = [gene1_full]
                else:
                    tr1_tr2[gene2] = keepTheGreatest(tr1_tr2[gene2], gene1_full)


def getResults(tr_list, gene):
    if gene in list(tr_list):
        return tr_list[gene]
    else:
        return "-"


_del = '\t'

with open("results_tr1.tsv", 'w') as R:
    R.write(f"tr1{_del}tr1_local{_del}tr2_across\n")
    for tr1_gene1, tr1_gene2 in tr1_tr1.items():
        R.write(f"{tr1_gene1}{_del}{tr1_gene2}{_del}{getResults(tr1_tr2, tr1_gene1)}\n")

with open("results_tr2.tsv", 'w') as R:
    R.write(f"tr2{_del}tr2_local{_del}tr1_across\n")
    for tr2_gene1, tr2_gene2 in tr2_tr2.items():
        R.write(f"{tr2_gene1}{_del}{tr2_gene2}{_del}{getResults(tr2_tr1, tr2_gene1)}\n")
